In [ ]:
!pip install goatools

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 11.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=75f533000f05a3ad261c19f979febe808747d97c51917e7d439c890d3f1ad0e8
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [ ]:
import os
import pickle
from goatools.base import download_go_basic_obo, download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.anno.genetogo_reader import Gene2GoReader
from goatools.cli.ncbi_gene_results_to_python import ncbi_tsv_to_py
from goatools.goea.go_enrichment_ns import GOEnrichmentStudyNS
from goatools.cli.ncbi_gene_results_to_python import ncbi_tsv_to_py

def download_data():
    #Downloads necessary data files
    !wget http://geneontology.org/ontology/go-basic.obo -O go-basic.obo
    !wget https://ftp.ncbi.nlm.nih.gov/gene/DATA/gene2go.gz -O gene2go.gz
    !gunzip gene2go.gz
    obo_fname = 'go-basic.obo'
    fin_gene2go = 'gene2go'
    return obo_fname, fin_gene2go

def create_background_gene_set(NCBI_file='protein coding human.txt'):
    output_py = 'genes_ncbi_9606_proteincoding.py'
    ncbi_tsv_to_py(NCBI_file, output_py)
    from genes_ncbi_9606_proteincoding import GENEID2NT
    background = []
    for gene in iter(sorted(GENEID2NT.values())):
      geneobj = sorted(gene._asdict().items())
      background.append(geneobj[2][1])

    print(len((set(background))))
    with open('background.pkl', 'wb') as f:
        pickle.dump(background, f)
    return background

def load_data(obo_fname, fin_gene2go, background_file='background.pkl'):
    #Loads ontologies, associations, and background gene set
    obodag = GODag(obo_fname)
    objanno = Gene2GoReader(fin_gene2go, taxids=[9606])
    ns2assoc = objanno.get_ns2assc()
    for nspc, id2gos in ns2assoc.items():
      print("{NS} {N:,} annotated human genes".format(NS=nspc, N=len(id2gos)))

    with open(background_file, 'rb') as f:
        background = pickle.load(f)
    background = list(map(int, background))

    return obodag, ns2assoc, background

def initialize_goea(background, ns2assoc, obodag):
    #Initializes a GOEA object
    goeaobj = GOEnrichmentStudyNS(
        background,
        ns2assoc,
        obodag,
        propagate_counts=False,
        alpha=0.05,
        methods=['fdr_bh']
    )
    return goeaobj

def load_study_genes(study_genes_file='entrez_lists.pkl'):
    #Loads study genes from a file
    with open(study_genes_file, 'rb') as f:
        dics = pickle.load(f)
    return dics

def run_goea(goeaobj, study_genes):
    #Runs GOEA for each phenotype and exports results to Excel
    for phenotype, gene_list in study_genes.items():
        geneids_study = list(map(int, gene_list))
        goea_results_all = goeaobj.run_study(geneids_study)
        goea_results_sig = [r for r in goea_results_all if r.p_fdr_bh < 0.05]
        goeaobj.wr_xlsx(f"{phenotype}.xlsx", goea_results_sig)

#Pipeline execution
obo_fname, fin_gene2go = download_data()
create_background_gene_set()
obodag, ns2assoc, background = load_data(obo_fname, fin_gene2go)
goeaobj = initialize_goea(background, ns2assoc, obodag)
study_genes = load_study_genes()
run_goea(goeaobj, study_genes)

--2025-03-06 16:44:38--  http://geneontology.org/ontology/go-basic.obo
Resolving geneontology.org (geneontology.org)... 34.233.67.155
Connecting to geneontology.org (geneontology.org)|34.233.67.155|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://geneontology.org/ontology/go-basic.obo [following]
--2025-03-06 16:44:38--  https://geneontology.org/ontology/go-basic.obo
Connecting to geneontology.org (geneontology.org)|34.233.67.155|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://purl.obolibrary.org/obo/go/go-basic.obo [following]
--2025-03-06 16:44:38--  http://purl.obolibrary.org/obo/go/go-basic.obo
Resolving purl.obolibrary.org (purl.obolibrary.org)... 172.64.150.197, 104.18.37.59, 2606:4700:4400::6812:253b, ...
Connecting to purl.obolibrary.org (purl.obolibrary.org)|172.64.150.197|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://current.geneontology.org/ontology/

In [ ]:
import pandas as pd
e = pd.read_excel('E.xlsx')

In [ ]:
e

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_fdr_bh,study_items
0,GO:0120162,BP,e,positive regulation of cold-induced thermogenesis,7/36,101/20596,4.102782e-10,6,7,0.000005,"154, 155, 7253, 7351, 7422, 10891, 133522"
1,GO:0043536,BP,e,positive regulation of blood vessel endothelia...,5/36,38/20596,5.881255e-09,8,5,0.000036,"3091, 3791, 4846, 7422, 23411"
2,GO:0045766,BP,e,positive regulation of angiogenesis,6/36,137/20596,1.282670e-07,7,6,0.000407,"358, 3091, 3791, 4846, 7422, 23411"
3,GO:1902894,BP,e,negative regulation of miRNA transcription,4/36,27/20596,1.340329e-07,11,4,0.000407,"3091, 4776, 5465, 7422"
4,GO:0009409,BP,e,response to cold,4/36,30/20596,2.085181e-07,4,4,0.000481,"154, 155, 7351, 7352"
5,GO:0001569,BP,e,branching involved in blood vessel morphogenesis,4/36,31/20596,2.391121e-07,7,4,0.000481,"3791, 4776, 5534, 7422"
6,GO:0043524,BP,e,negative regulation of neuron apoptotic process,6/36,156/20596,2.767870e-07,7,6,0.000481,"3091, 3791, 4776, 7351, 10891, 23411"
7,GO:0010629,BP,e,negative regulation of gene expression,7/36,266/20596,3.357390e-07,8,7,0.000510,"639, 1026, 1636, 3091, 3791, 7422, 23411"
8,GO:0097746,BP,e,blood vessel diameter maintenance,4/36,36/20596,4.448623e-07,6,4,0.000601,"154, 186, 1636, 4846"
9,GO:0001666,BP,e,response to hypoxia,6/36,173/20596,5.088891e-07,5,6,0.000619,"3091, 5465, 7019, 7351, 7352, 7422"
